In [1]:
import datetime
import psutil
import multiprocessing        as mp
from   multiprocessing    import Manager
from   multiprocessing    import Pool
mgr                            = Manager()
ns                             = mgr.Namespace()
import numpy                  as np
import pandas                 as pd
from   time               import sleep
from   time               import time

print('This Machine Has', psutil.cpu_count(), 'Cores')
print('For parameter    pool = Pool(nn)     ensure (nn) is ', psutil.cpu_count()-1, ' or less' )

This Machine Has 16 Cores
For parameter    pool = Pool(nn)     ensure (nn) is  15  or less


In [2]:
def multi_run_wrapper(args):
    return compare_adjacent_matrix_cells(*args)

def compare_adjacent_matrix_cells(ns, row_pointer, df_csv_row_count, first_numeric_row): 
    
    function_return_list         = []
    color_values_list            = []
    color_pairs_list             = []
    
    lock.acquire()
    
    df_inside_csv_file = pd.DataFrame(ns.numpy_csv_file)
    
    sleep(0.03)
    
    lock.release()
    
    matrix_max_row_pointer       = df_csv_row_count
    matrix_max_column_pointer    = len(df_inside_csv_file.columns)
    column_pointer               = 0
    column_pointer_minus1        = 0
    row_pointer_minus1           = 0
    cell_value_compare_up        = 0
    cell_value_compare_left      = 0
    value_start_row              = 0
    
    try:
        if  df_inside_csv_file.iloc[row_pointer,0].isnumeric():

            if value_start_row                   == 0:
                value_start_row                   = row_pointer

            try:
                for column_pointer in range(matrix_max_column_pointer): 

                    column_pointer_minus1 = column_pointer - 1

                    cell_value_delta_up           = 0
                    cell_value_delta_left         = 0
                    cell_value_anchor             = int(df_inside_csv_file.iloc[row_pointer,column_pointer])

                    if row_pointer >= first_numeric_row + 1:
                        row_pointer_minus1                = row_pointer - 1  

                        cell_value_compare_up       = df_csv_file.iloc[row_pointer_minus1, column_pointer] 

                        if cell_value_anchor       != cell_value_compare_up:
                            cell_value_delta_up     = int(cell_value_compare_up)

                    try:
                        if column_pointer > 0:
                            cell_value_compare_left  = df_csv_file.iloc[row_pointer, column_pointer_minus1] 

                            if cell_value_anchor       != cell_value_compare_left:
                                cell_value_delta_left   = int(cell_value_compare_left)

                    except Exception as e:
                        print ("Error @ if column_pointer > 0:")
                        print(e)

                    function_return_list.append([row_pointer, 
                                                 column_pointer, 
                                                 cell_value_anchor, 
                                                 cell_value_delta_up, 
                                                 cell_value_delta_left])
                    
                    if cell_value_anchor > 0:
                        color_values_list.append(cell_value_anchor)
                    
                        if cell_value_delta_up   > 0:
                            if cell_value_anchor > cell_value_delta_up:
                                color_pairs_list.append([cell_value_anchor,
                                                         cell_value_delta_up])
                            if cell_value_anchor < cell_value_delta_up:
                                color_pairs_list.append([cell_value_delta_up,
                                                         cell_value_anchor])
                        if cell_value_delta_left > 0:
                            if cell_value_anchor > cell_value_delta_left:
                                color_pairs_list.append([cell_value_anchor,
                                                         cell_value_delta_left])
                            if cell_value_anchor < cell_value_delta_left:
                                color_pairs_list.append([cell_value_delta_left,
                                                         cell_value_anchor])                            

                    sleep(0.01)

            except Exception as e:
                print('Bummer: Column Pointer Set Error')
                print(e)
                print('row_pointer', row_pointer)
                print('column_pointer', column_pointer)
                
    except Exception as e:
        print('Bummer: Row Pointer Set Error')
        print(e)
        print('row_pointer:', row_pointer)
        print('df_inside_csv_file.iloc[row_pointer,0]', df_inside_csv_file.iloc[row_pointer,0])
        
    print('Row', row_pointer, 'Complete @', ('20' + datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")))
    
    return function_return_list, color_values_list, color_pairs_list
    

In [46]:
csv_file = 's3://chameleon-student-lab-files/SageMaker and Multiprocessing/multiprocessing-student-lab-numeric-matrix-100-Rows.csv'

if __name__ == "__main__":
    print('Execution Begins (Parse Matrix): ', ('20' + datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")))
    print(' ')
    print('Analyzing File: \n', csv_file)
    print(' ')
    
    
    collect_color_values_list = []
    collect_color_pairs_list  = []
    
    start_time                = time()
    
    df_csv_file               = pd.read_csv(csv_file,
                                            low_memory=False,
                                            header=None,
                                            index_col=False)
    df_csv_row_count          = len(df_csv_file)
    df_csv_column_count       = len(df_csv_file.columns)
    
    ns.numpy_csv_file         = df_csv_file.to_numpy()
    
    for idx in range(df_csv_row_count):
        if df_csv_file.iloc[idx,0].isnumeric():
            first_numeric_row = idx
            break
            
    lock    = mp.Lock()
    pool    = Pool(15)
    
    results = pool.map(multi_run_wrapper,[(ns, 
                                           row_pointer, 
                                           df_csv_row_count, 
                                           first_numeric_row) for row_pointer in range(df_csv_row_count)])
    
    df_results = pd.DataFrame(results)

    pool.close()
    
    print(' ')
    print('Collecting Distinct Color Proportions for Values > 0 . . . . . .')
    print(' ')
    
    for result in df_results[first_numeric_row:][1]:
        for x in result:
            collect_color_values_list.append(x)

    numpy_collect_color_values = np.asarray(collect_color_values_list)
    color_value_count = len(numpy_collect_color_values)

    (unique, counts) = np.unique(numpy_collect_color_values, return_counts=True)
    frequencies = np.asarray((unique, counts)).T
    for value, count in frequencies:
        print(value, '=', str((count/color_value_count)*100) + ' %')

    print(' ')
    print(' Collecting Color Pair Counts for Values > 0 . . . . . . . . . .')
    print(' ')
    
    for result in df_results[first_numeric_row:][2]:
        for x,y in result:
            collect_color_pairs_list.append(str(x) + ':' + str(y))
    numpy_collect_color_pairs = np.asarray(collect_color_pairs_list)

    (unique, counts) = np.unique(numpy_collect_color_pairs, return_counts=True)
    frequencies = np.asarray((unique, counts)).T
    print(' ')
    print(frequencies)
    print(' ')
    
    end_time = time()
    print(' ')
    print('Execution Complete - Total Elapsed time (Minutes):', ((end_time-start_time)/60))


Execution Begins (Parse Matrix):  2021-09-05-19-18-19
 
Analyzing File: 
 s3://chameleon-student-lab-files/SageMaker and Multiprocessing/multiprocessing-student-lab-numeric-matrix-100-Rows.csv
 
Row 0 Complete @ 2021-09-05-19-18-20
Row 2 Complete @ 2021-09-05-19-18-20
Row 4 Complete @ 2021-09-05-19-18-20
Row 1 Complete @ 2021-09-05-19-18-21
Row 3 Complete @ 2021-09-05-19-18-21
Row 5 Complete @ 2021-09-05-19-18-21
Row 6 Complete @ 2021-09-05-19-18-30
Row 8 Complete @ 2021-09-05-19-18-30
Row 12 Complete @ 2021-09-05-19-18-30
Row 10 Complete @ 2021-09-05-19-18-30
Row 14 Complete @ 2021-09-05-19-18-30
Row 16 Complete @ 2021-09-05-19-18-30
Row 18 Complete @ 2021-09-05-19-18-30
Row 22 Complete @ 2021-09-05-19-18-31
Row 20 Complete @ 2021-09-05-19-18-31
Row 26 Complete @ 2021-09-05-19-18-31
Row 24 Complete @ 2021-09-05-19-18-31
Row 28 Complete @ 2021-09-05-19-18-31
Row 30 Complete @ 2021-09-05-19-18-31
Row 32 Complete @ 2021-09-05-19-18-31
Row 34 Complete @ 2021-09-05-19-18-31
Row 7 Complete 

In [36]:
#-- First of Three Returned List (function_return_list)

print(results[62][0][12:13])  #-- Yellow Shaded (M63)
print(results[63][0][12:13])  #-- Yellow Shaded (M64)

[[62, 12, 1, 0, 0]]
[[63, 12, 3, 1, 0]]


In [43]:
#-- Second of Three Returned List (color_values_list)

print(results[62][1])
print(results[62][1][12:13]) #-- Green Shaded (Y63)

print(results[63][1])
print(results[63][1][12:13]) #-- Green Shaded (U64)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[3]


In [45]:
#-- Third of Three Returned Lists (color_pairs_list)

print(results[62][2])
print(results[62][2][12:13])  #-- EMPTY VALUE HERE . . . Since there is only one color change, the [12:13] is out of range 
print(results[62][2][0:1])    #-- Red Square (X63 and Y63) . . . Only a single item exists in this list
print(' ')

print(results[63][2])         #-- Black Square (I64 to Y64)
print(results[63][2][12:13])  #-- Red Shaded (X64 and Y64)

[[2, 1]]
[]
[[2, 1]]
[[3, 1], [3, 1], [3, 1], [3, 1], [3, 1], [3, 1], [3, 1], [3, 1], [3, 1], [3, 1], [3, 1], [3, 1], [3, 2]]
[[3, 2]]
